# 作业（三）

## 独热编码和哑变量之间有什么区别？

答：本质上差不多，哑变量常用于回归模型，one-hot在任何模型都可以用；
哑变量为pandas的get_dummy方法，one-hot 在sklearn中引用（from sklearn.preprocessing import OneHotEncoder）；
数据量大的话，哑变量太消耗资源，one-hot可以读数组，因此大规模数据集很方便。

## 为什么要进⾏特征归⼀化？（简单描述即可）

答：特征的单位或者大小相差较大，或者某特征的方差相比其他的特征要大出几个数量级，容易影响（支配）目标结果，使得一些算法无法学习到其它的特征， 需要用到一些方法进行无量纲化，使不同规格的数据转换到同一规格。

## 填补缺失值的时候（中位数、众数、均值）三种填补⽅式的应⽤场景。

答：中位数：数据比较离散时，如：在评估一个地区收入水平时，用中位数；
众数:行为时间点等，如：“购物时间”；
均值：商品推荐场景，如：“年收入”。

# 作业（四）

In [1]:
# import 工具库
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 加载数据集
df_train = pd.read_csv('./train.csv')

.以下对数据进行预处理

In [3]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)# axis= 0 代表列

df_train['Age'] = imp.fit_transform(df_train[['Age']].values) # Age空值处理,使用平均值

df_train['Embarked'].fillna("S",inplace=True) # Embarked空值处理，取众数“S”

df_train.drop(['Cabin','PassengerId','Name','Ticket'],axis=1,inplace=True) # Cabin太多空值，删除该行;PassengerId\Name\Ticket为用户唯一标签，无用删除

df_train['Sex'] = df_train['Sex'].map({"male":1,"female":0}) #性别按男：1 女：0替换标识

df_train.loc[:,'family_size'] = df_train['SibSp']+df_train['Parch']+1 # 增多一列家庭成员列 1代表自己

embarked = pd.get_dummies(df_train['Embarked'],prefix="embarked_") # embarked做哑变量变化
df_train = pd.concat([df_train,embarked],axis=1)

df_train.drop(['SibSp','Parch','Embarked'],axis=1,inplace=True) # 删除已处理后不需要的列

.以下对数据进行标准化处理

In [4]:
X = df_train.iloc[:,1:]
Y = np.array(df_train.iloc[:,0])

X = X.as_matrix().astype(np.float) # 所有特征转化为np.float

from sklearn.preprocessing import StandardScaler #标准化库

scaler = StandardScaler() #标准化

X = scaler.fit_transform(X)

In [5]:
X.shape

(891, 8)

.以下使用交叉验证，比较两个算法的准确率

In [6]:
#调入工具包
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,KFold
import matplotlib.pyplot as plt

# 初始化模型
models = []
models.append(('KNN',KNeighborsClassifier()))
models.append(('NB',GaussianNB()))

# 初始化
results = []
names = []
scoring = 'accuracy' # 准确率
for name, model in models:
    kfold = KFold(5, shuffle=True, random_state = 0)
    cv_results = cross_val_score(model, X, Y, cv=kfold)
    results.append(cv_results) # 交叉验证给的结果分
    names.append(name)
    # 模型的标准差，体现模型的分值波动，std越小越稳定
    msg = "%s: %f (%f)" %(name, cv_results.mean(), cv_results.std())
    print(msg)
    print("-------------------------")
fig = plt.figure()
fig.suptitle("Algorithm Comparison")
ax = fig.add_subplot(111)

plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

KNN: 0.803584 (0.029128)
-------------------------
NB: 0.785607 (0.019851)
-------------------------


<Figure size 640x480 with 1 Axes>

总结：就准确率比较来说，KNN要比NB数据表现要好。